# Analyse von Änderungstypen in Builds

In [54]:
import dask.dataframe as dd
import pandas as pd

def sloc_percent(row):
    row['sloc_percent'] = 100*(row.git_diff_src_churn.astype('float')/row.gh_sloc.astype('float'))
    row['gh_test_lines_per_kloc'] = row.gh_test_lines_per_kloc/1000
    return row

#df = dd.read_csv('data/travistorrent_11_1_2017.csv',blocksize=25000000,usecols=['tr_build_id','gh_project_name','git_num_all_built_commits','gh_is_pr','git_trigger_commit','gh_sloc','git_diff_src_churn','gh_diff_files_added','gh_diff_files_deleted','gh_diff_files_modified'])
df = dd.read_csv('data/travistorrent_11_1_2017.csv',blocksize=25000000,usecols=['tr_build_id','gh_project_name','git_num_all_built_commits','git_trigger_commit','gh_sloc','git_diff_src_churn', 'gh_test_lines_per_kloc','gh_diff_src_files','gh_diff_doc_files','gh_diff_other_files','git_diff_test_churn'])
pdf = df.drop_duplicates().map_partitions(sloc_percent).compute()

## Änderungstypen von Builds

### Source Code Änderungen

#### Zeilenbasiert

In [55]:
pdf_greater_eq_one_line_change = pdf[pdf.git_diff_src_churn >= 1]
pdf_greater_eq_one_line_change['git_diff_src_churn'].describe()

count    5.422920e+05
mean     2.030520e+02
std      8.480826e+03
min      1.000000e+00
25%      4.000000e+00
50%      1.400000e+01
75%      6.000000e+01
max      5.278263e+06
Name: git_diff_src_churn, dtype: float64

In [56]:
(5.422920e+05/float(680209))*100

79.72432002516874

Ergebnis: 80% der Builds enthalten mind. eine zeilenbasierte Source Code Änderung.

#### Dateibasiert

In [57]:
pdf_one_file_change = pdf[pdf.gh_diff_src_files >= 1]
#pdf_one_line_change.sort_values('gh_sloc',inplace=True,ascending=False)
pdf_one_file_change['gh_diff_src_files'].describe()

count    580356.000000
mean          6.825200
std          71.596903
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max       37918.000000
Name: gh_diff_src_files, dtype: float64

In [58]:
(580356/float(680209))*100

85.32024715932897

Ergebnis: 85% der Builds enthalten mind. eine dateibasierte Source Code Änderung.

In [59]:
580356-5.422920e+05

38064.0

Ergebnis: ca. 40k Builds mehr mit Source Code Dateiänderungen als mit Source Code Zeilenänderungen. Warum?! 

Erklärung:

1. Umbenennung einer Datei
1. Rechteänderung

### Builds ohne Source Code Änderung

#### Zeilenbasiert

In [60]:
680209-5.422920e+05

137917.0

In [61]:
137917.0/680209*100

20.275679974831267

Ergebnis: 20% der Builds haben keine zeilenbasierten Änderungen

#### Dateibasiert

In [62]:
680209-580356

99853

In [63]:
99853/680209*100

14.679752840671028

Ergebnis: 15% der Builds haben keine dateibasierten Änderungen

### Builds mit in Doc Änderungen (Dateibasiert)

In [64]:
pdf_one_doc_file_change = pdf[pdf.gh_diff_doc_files >= 1]
pdf_one_doc_file_change['gh_diff_doc_files'].describe()

count    60706.000000
mean         5.135489
std         31.128898
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       2769.000000
Name: gh_diff_doc_files, dtype: float64

In [65]:
60706/float(680209)*100

8.924609936063769

Ergebnis: 9% der Builds sind Änderungen in der Dokumentation.

TODO: Wieviele Projekte haben Dokumentation in Git?

### Builds mit in Other Änderungen (Dateibasiert)

In [73]:
pdf_one_other_file_change = pdf[pdf.gh_diff_other_files >= 1]
pdf_one_other_file_change['gh_diff_other_files'].describe()

count    195712.000000
mean          4.904390
std          56.068927
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max       19202.000000
Name: gh_diff_other_files, dtype: float64

In [75]:
195712/float(680209)*100

28.77233320935183

Ergebnis: 29% der Builds sind Änderungen in sonstigen Dateien.

TODO: Wie ist Other definiert?

## Build mit 1-Zeilen Änderungen

In [71]:
pdf_one_line_change = pdf[pdf.git_diff_src_churn == 1]
#pdf_one_line_change.sort_values('gh_sloc',inplace=True,ascending=False)
pdf_one_line_change['git_diff_src_churn'].describe()

count    16159.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: git_diff_src_churn, dtype: float64

In [70]:
16159/float(680209)*100 # all builds (Referenzzahlen übernommen von CommitsPerBuild.ipynb)

2.375593383798215

Ergebnis: 2.4% aller Builds sind 1-Zeilen Source Code Änderungen.

In [69]:
16159/float(557211)*100 # one commit builds (Referenzzahlen übernommen von CommitsPerBuild.ipynb)

2.8999786436376884

Ergebnis: 2.9% aller 1-Commit Builds sind 1-Zeilen Source Code Änderungen.

In [ ]:
16159/float(5.422920e+05)*100 # all builds with greater or eq 1 src change

Ergebnis: 3% aller Builds mit Source Code Änderungen (mind. 1 Zeile), sind 1-Zeilen Source Code Änderungen.